Download the necessary libraries & import data.

Link to data - https://raw.githubusercontent.com/DataSlingers/clustRviz/master/data/authors.rda

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD

from sklearn.decomposition import FastICA, NMF, KernelPCA, PCA
from scipy.spatial.distance import pdist, squareform
from sklearn.manifold import SpectralEmbedding
import seaborn as sns
import umap

from adjustText import adjust_text

In [3]:
df = pd.read_csv('authors.csv').rename(columns = {'Unnamed: 0': 'Author'})
X = df.copy().drop(['Author','BookID'], axis=1)


FileNotFoundError: [Errno 2] No such file or directory: 'authors.csv'

In [ ]:
X_transpose = X.T
X_transpose = X_transpose.rename(columns = {i:f'Chapter{i}' for i in range(X_transpose.shape[1])})
X_transpose

In [ ]:
# Add a small constant to avoid zeros (NMF needs non-negative data)
X_nmf = X_transpose + 1e-6

# Run NMF with k topics
k = 4
model = NMF(n_components=k, init='random', random_state=0, max_iter=500)
W = model.fit_transform(X_nmf)  # shape: (69 words, k topics)
H = model.components_           # shape: (k topics, 841 chapters)

# Turn W into a DataFrame with word labels
W_df = pd.DataFrame(W, index=X_transpose.index, columns=[f'Topic{i+1}' for i in range(k)])
W_df

In [ ]:
# NOW LETS INTERPRET OUR TOPICS
top_words_per_topic = {}
n = 10
for topic in W_df.columns:
    top_words = W_df[topic].sort_values(ascending=False).head(n)
    top_words_per_topic[topic] = top_words.index.tolist()

summary_df = pd.DataFrame(top_words_per_topic)
summary_df #importance in ascending order!

Topic 1 = Narritive/Descriptive (past tense)

Topic 2 = Reflective/Personal voice

Topic 3 = Relational

Topic 4 = Dialogue/Character-driven phrasing

In [ ]:
plt.figure(figsize=(10, 7))
plt.scatter(W[:, 0], W[:, 1], alpha=0.7)

for i, word in enumerate(X_transpose.index):
    plt.text(W[i, 0], W[i, 1], word, fontsize=8)

plt.xlabel("Topic 1 weight")
plt.ylabel("Topic 2 weight")
plt.title("Words in NMF Topic Space")
plt.show()

This is hard to read so we will take out a lot of the insignificant words and replot.

In [ ]:
top_n = n
top_words = set()
for topic in W_df.columns:
    top_words.update(W_df[topic].sort_values(ascending=False).head(top_n).index)
W_top = W_df.loc[list(top_words)] # Filter W to only top words

plt.figure(figsize=(10, 7))
plt.scatter(W_top.iloc[:, 0], W_top.iloc[:, 1], alpha=0.7)

texts = []
for i, word in enumerate(W_top.index):
    texts.append(plt.text(W_top.iloc[i, 0], W_top.iloc[i, 1], word, fontsize=8))

adjust_text(texts, arrowprops=dict(arrowstyle='-', color='gray', lw=0.5))
plt.xlabel("Topic 1 weight")
plt.ylabel("Topic 2 weight")
plt.title("Top Words in NMF Topic Space")
plt.show()
